# Aerospike Python Client Tutorial 
### Refer to https://www.aerospike.com/docs/client/python/index.html for information on installing the Aerospike Python client.
#### Tested with Python 3.7


In [3]:
# IP Address or DNS name for one host in your Aerospike cluster
AS_HOST ="127.0.0.1"

# Please reach out to us if you do not have a feature key
AS_FEATURE_KEY_PATH = "/etc/aerospike/features.conf"

AS_PORT = 3000 # Usually 3000, but change here if not

In [4]:
import aerospike

## Create Sample Data and load it into Aerospike

In [5]:
# We create age vs salary data, using three different Gaussian distributions
import numpy as np
import pandas as pd
import math

# Create covariance matrix from std devs + correlation
def covariance_matrix(std_dev_1,std_dev_2,correlation):
    return [[std_dev_1 ** 2, correlation * std_dev_1 * std_dev_2], 
           [correlation * std_dev_1 * std_dev_2, std_dev_2 ** 2]]

# Return a bivariate sample given means/std dev/correlation
def age_salary_sample(distribution_params,sample_size):
    mean = [distribution_params["age_mean"], distribution_params["salary_mean"]]
    cov = covariance_matrix(distribution_params["age_std_dev"],distribution_params["salary_std_dev"],
                            distribution_params["age_salary_correlation"])
    return np.random.multivariate_normal(mean, cov, sample_size).T

# Define the characteristics of our age/salary distribution
age_salary_distribution_1 = {"age_mean":25,"salary_mean":50000,
                             "age_std_dev":1,"salary_std_dev":5000,"age_salary_correlation":0.3}

age_salary_distribution_2 = {"age_mean":45,"salary_mean":80000,
                             "age_std_dev":4,"salary_std_dev":10000,"age_salary_correlation":0.7}

age_salary_distribution_3 = {"age_mean":35,"salary_mean":70000,
                             "age_std_dev":2,"salary_std_dev":9000,"age_salary_correlation":0.1}

distribution_data = [age_salary_distribution_1,age_salary_distribution_2,age_salary_distribution_3]

# Sample age/salary data for each distributions
group_1_ages,group_1_salaries = age_salary_sample(age_salary_distribution_1,sample_size=100)
group_2_ages,group_2_salaries = age_salary_sample(age_salary_distribution_2,sample_size=120)
group_3_ages,group_3_salaries = age_salary_sample(age_salary_distribution_3,sample_size=80)

ages=np.concatenate([group_1_ages,group_2_ages,group_3_ages])
salaries=np.concatenate([group_1_salaries,group_2_salaries,group_3_salaries])

print("Data created")

Data created


In [6]:
# Turn the above records into a Data Frame
# First of all, create an array of arrays
inputBuf = []

for  i in range(0, len(ages)) :
     id = i + 1 # Avoid counting from zero
     name = "Individual: {:03d}".format(id)
     # Note we need to make sure values are typed correctly
     # salary will have type numpy.float64 - if it is not cast as below, an error will be thrown
     age = float(ages[i])
     salary = int(salaries[i])
     inputBuf.append((id, name,age,salary))

for i in inputBuf:
    print (i)



(1, 'Individual: 001', 24.304499333848025, 46865)
(2, 'Individual: 002', 25.127370725646376, 58731)
(3, 'Individual: 003', 24.433587308486384, 39822)
(4, 'Individual: 004', 23.466317614770414, 46800)
(5, 'Individual: 005', 25.901554554617075, 44622)
(6, 'Individual: 006', 25.244089945979592, 48962)
(7, 'Individual: 007', 25.513546531713885, 54015)
(8, 'Individual: 008', 24.384542542073284, 48850)
(9, 'Individual: 009', 23.969228059245417, 49939)
(10, 'Individual: 010', 25.32659498864832, 52270)
(11, 'Individual: 011', 24.52188955913116, 56682)
(12, 'Individual: 012', 25.307831842818935, 48937)
(13, 'Individual: 013', 25.25646020804283, 54880)
(14, 'Individual: 014', 25.678511730398245, 50121)
(15, 'Individual: 015', 24.05942103892467, 60348)
(16, 'Individual: 016', 25.749046404565952, 53522)
(17, 'Individual: 017', 22.897339115317486, 41738)
(18, 'Individual: 018', 25.85659550137486, 51606)
(19, 'Individual: 019', 25.344785148917193, 47040)
(20, 'Individual: 020', 23.821932401631045, 5

## Connect to the Aerospike

In [12]:
# Configure the client
config = {
  'hosts': [ (AS_HOST, AS_PORT) ]
}

# Create a client and connect it to the cluster
try:
  client = aerospike.client(config).connect()
except:
  import sys
  print("failed to connect to the cluster with", config['hosts'])
  sys.exit(1)

## Write Data

In [8]:
# Type 'show namespaces' at the aql prompt if you are not sure about the namespace
namespace= "test"
write_set= "write_set"

for  i in inputBuf:
  _id, _name, _age, _salary = i
  key = (namespace, write_set,_id)
  client.put(key, {'id': _id,'name': _name,'age': _age,'salary': _salary})

## Read Data

In [9]:
for  i in inputBuf:
    _id, _name, _age, _salary = i
    key = (namespace, write_set,_id)
    (key, metadata, record) = client.get(key)
    print(record)


{'id': 1, 'name': 'Individual: 001', 'age': 24.304499333848025, 'salary': 46865}
{'id': 2, 'name': 'Individual: 002', 'age': 25.127370725646376, 'salary': 58731}
{'id': 3, 'name': 'Individual: 003', 'age': 24.433587308486384, 'salary': 39822}
{'id': 4, 'name': 'Individual: 004', 'age': 23.466317614770414, 'salary': 46800}
{'id': 5, 'name': 'Individual: 005', 'age': 25.901554554617075, 'salary': 44622}
{'id': 6, 'name': 'Individual: 006', 'age': 25.244089945979592, 'salary': 48962}
{'id': 7, 'name': 'Individual: 007', 'age': 25.513546531713885, 'salary': 54015}
{'id': 8, 'name': 'Individual: 008', 'age': 24.384542542073284, 'salary': 48850}
{'id': 9, 'name': 'Individual: 009', 'age': 23.969228059245417, 'salary': 49939}
{'id': 10, 'name': 'Individual: 010', 'age': 25.32659498864832, 'salary': 52270}
{'id': 11, 'name': 'Individual: 011', 'age': 24.52188955913116, 'salary': 56682}
{'id': 12, 'name': 'Individual: 012', 'age': 25.307831842818935, 'salary': 48937}
{'id': 13, 'name': 'Individ

{'id': 224, 'name': 'Individual: 224', 'age': 38.268952445865935, 'salary': 60153}
{'id': 225, 'name': 'Individual: 225', 'age': 33.91228381267188, 'salary': 77378}
{'id': 226, 'name': 'Individual: 226', 'age': 38.34715952661958, 'salary': 81699}
{'id': 227, 'name': 'Individual: 227', 'age': 35.5925322009585, 'salary': 72465}
{'id': 228, 'name': 'Individual: 228', 'age': 34.58190977648209, 'salary': 66225}
{'id': 229, 'name': 'Individual: 229', 'age': 37.68327548633769, 'salary': 57314}
{'id': 230, 'name': 'Individual: 230', 'age': 36.122309523791834, 'salary': 61779}
{'id': 231, 'name': 'Individual: 231', 'age': 36.10591686948013, 'salary': 72038}
{'id': 232, 'name': 'Individual: 232', 'age': 32.353142028555105, 'salary': 59520}
{'id': 233, 'name': 'Individual: 233', 'age': 35.68739011715068, 'salary': 62823}
{'id': 234, 'name': 'Individual: 234', 'age': 36.484734910946614, 'salary': 76148}
{'id': 235, 'name': 'Individual: 235', 'age': 34.24028050138242, 'salary': 58137}
{'id': 236, '